In [ ]:
# Travailler sur Google Colab
from google.colab import drive
import os
import numpy as np
import tensorflow as tf
import cv2
import shutil
import matplotlib.pyplot as plt
import requests
import pandas as pd

In [ ]:
%pip install tensorflow_hub
import tensorflow_hub as hub

In [ ]:
# Monter Google Drive
drive.mount('/content/drive')

In [ ]:
# Chargement du modèle de détection
print("Chargement du modèle de détection...")
detector = hub.load("https://tfhub.dev/tensorflow/ssd_mobilenet_v2/fpnlite_320x320/1")

In [ ]:
# Chargement des noms de classes
classe = np.array([
    'background', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
    'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant',
    'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse',
    'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack',
    'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis',
    'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove',
    'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass',
    'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
    'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
    'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
    'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
    'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink',
    'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear',
    'hair drier', 'toothbrush'])
classes = np.concatenate([classe, np.repeat('none', 100)])

In [ ]:
# Dossier contenant les images à traiter
dossier_images = "/content/drive/MyDrive/Colab Notebooks/images"
image_files = [f for f in os.listdir(dossier_images) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]


In [ ]:
# Stockage des résultats
detection_results = []

In [ ]:
# Traitement par lot des images
for filename in image_files:
    path = os.path.join(dossier_images, filename)
    image = cv2.imread(path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

In [ ]:
image = cv2.imread(path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

In [ ]:
input_tensor = tf.convert_to_tensor(image)
input_tensor = input_tensor[tf.newaxis, ...]
detections = detector(input_tensor)

In [ ]:
height, width, _ = image.shape

for i in range(len(scores)):
    if scores[i] < 0.3:
        continue

        box = boxes[i] * np.array([height, width, height, width])
        box = box.astype(np.int32)
               
        # Ajouter au DataFrame
        detection_results.append({
            "image": filename,
            "label": classes[labels[i]],
            "score": scores[i],
            "box": box.tolist()
        })
        
        # Dessiner la boîte englobante sur l'image
        cv2.rectangle(image, (box[1], box[0]), (box[3], box[2]), (0, 255, 0), 2)
        plt.text(box[1], box[0] - 10, f"{classes[labels[i]]} : {scores[i]*100:.2f}%%", color='red')

    plt.figure(figsize=(6, 6))
    plt.imshow(image)
    plt.title(f"Détections : {filename}")
    plt.axis('off')
    plt.show()

In [ ]:
# Création du DataFrame avec les détections
df_detections = pd.DataFrame(detection_results)
print(df_detections.head())

In [ ]:
# Enregistrer en CSV
df_detections.to_csv("/content/drive/MyDrive/detection_results.csv", index=False)
print("CSV enregistré sur Google Drive.")

In [ ]:
# Classification avec ViT pour toutes les images
!pip install transformers
from transformers import ViTImageProcessor, ViTForImageClassification
from PIL import Image as PILImage

processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224')
model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224')

vit_results = []

for filename in image_files:
    image_path = os.path.join(dossier_images, filename)
    image_pil = PILImage.open(image_path)

    inputs = processor(images=image_pil, return_tensors="pt")
    outputs = model(**inputs)
    logits = outputs.logits
    predicted_class_idx = logits.argmax(-1).item()
    label = model.config.id2label[predicted_class_idx]

    vit_results.append({"image": filename, "ViT_label": label})

    # Affichage
    plt.imshow(image_pil)
    plt.axis('off')
    plt.title(f"ViT: {label}")
    plt.show()

In [ ]:
# Enregistrement des prédictions ViT
df_vit = pd.DataFrame(vit_results)
df_vit.to_csv("/content/drive/MyDrive/vit_classification_results.csv", index=False)
print("Résultats ViT enregistrés sur Google Drive.")